In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score, mean_absolute_percentage_error

In [15]:
scenarios = {"optimistic":"ssp1_2_6",
             "neutral":"ssp2_4_5",
             "pessimistic":"ssp5_8_5"}

In [16]:
scenario = scenarios["optimistic"]
df = pd.read_csv(f"../data/weather_agg_{scenario}.csv")

In [17]:
df.sort_values("year", ascending = True)

,scenario,department,code_dep,year,mean_temp,std_temp,mean_max_temp,std_max_temp,total_precip,std_precip,yield
0,historical,Ain,1,1982,282.77924,7.090540,287.48267,8.055373,0.027494,0.000120,3.950078
132,historical,Alpes_de_Haute_Provence,4,1982,278.38060,6.779194,283.29404,7.257900,0.030277,0.000157,3.095108
2211,historical,Orne,61,1982,282.96463,6.698862,287.08145,7.842164,0.033426,0.000147,5.022091
66,historical,Allier,3,1982,283.21872,7.079867,288.41577,8.097082,0.023859,0.000113,3.638415
462,historical,Cantal,15,1982,280.22562,6.335462,284.73715,7.335979,0.053492,0.000230,2.297826
...,...,...,...,...,...,...,...,...,...,...,...
3692,ssp1_2_6,Cotes_d_Armor,22,2050,285.33655,4.968215,288.33340,5.541448,0.067060,0.000280,NaN
5132,ssp1_2_6,Meurthe_et_Moselle,54,2050,284.37497,7.256599,288.55673,8.251174,0.032957,0.000146,NaN
3584,ssp1_2_6,Cher,18,2050,285.50510,7.016250,289.92963,7.843265,0.040932,0.000188,NaN
6140,ssp1_2_6,Yvelines,78,2050,285.41495,6.969815,289.36765,7.847225,0.009245,0.000047,NaN


In [18]:
df.head()

,scenario,department,code_dep,year,mean_temp,std_temp,mean_max_temp,std_max_temp,total_precip,std_precip,yield
0,historical,Ain,1,1982,282.77924,7.090540,287.48267,8.055373,0.027494,0.000120,3.950078
1,historical,Ain,1,1983,281.72400,7.357678,286.23380,8.096182,0.029012,0.000119,2.648276
2,historical,Ain,1,1984,282.05420,6.998279,286.64294,8.002853,0.027781,0.000125,4.822581
3,historical,Ain,1,1985,281.67966,7.407989,286.12997,8.244929,0.034206,0.000145,4.196774
4,historical,Ain,1,1986,282.42148,6.688269,287.31296,7.404187,0.023955,0.000116,3.598450


In [19]:
df_train = df[df["year"]<2005]
df_val = df[(df["year"]<2019) & (df["year"]>=2005)]
df_test = df[df["year"]>=2019]

In [23]:
meta_val = df_val[["year", "department"]].reset_index(drop=True)

X_train = df_train.drop(columns = ["scenario", "year","yield"])
X_val = df_val.drop(columns = ["scenario", "year","yield"])
X_test = df_test.drop(columns = ["scenario", "year","yield"])
y_train = df_train["yield"]
y_val = df_val["yield"]

In [24]:
X_train.head()

,department,code_dep,mean_temp,std_temp,mean_max_temp,std_max_temp,total_precip,std_precip
0,Ain,1,282.77924,7.090540,287.48267,8.055373,0.027494,0.000120
1,Ain,1,281.72400,7.357678,286.23380,8.096182,0.029012,0.000119
2,Ain,1,282.05420,6.998279,286.64294,8.002853,0.027781,0.000125
3,Ain,1,281.67966,7.407989,286.12997,8.244929,0.034206,0.000145
4,Ain,1,282.42148,6.688269,287.31296,7.404187,0.023955,0.000116


In [25]:
cat_col = ["department"]
num_col = X_train.select_dtypes(include = "number").columns.tolist()
preprocessor = ColumnTransformer(transformers = [("cat", OneHotEncoder(drop="first"), cat_col),
                                                 ("num", StandardScaler(), num_col)],
                                 remainder="passthrough")
model = LinearRegression()
pipe = make_pipeline(preprocessor, model)
pipe.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('columntransformer', ...), ('linearregression', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('cat', ...), ('num', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'passthrough'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the di

In [26]:
y_pred_val = pipe.predict(X_val)

r_2 = r2_score(y_val, y_pred_val)
mape = mean_absolute_percentage_error(y_val, y_pred_val)

In [27]:
print(r_2)
print(mape)

0.5958159514047175
0.11519282354065172


In [29]:
pred_val_df = pd.DataFrame({
    "year": df_val["year"].values,
    "department": df_val["department"].values,
    "actual_yield": y_val.values,
    "predicted_yield": y_pred_val
})


Let's look at our prediction for a specific region

In [30]:
region_choice = "Ain"

In [35]:
region_perf = pred_val_df[
    pred_val_df["department"] == region_choice
].reset_index()

region_perf = (
    region_perf
    .groupby("year")
    .agg(
        actual=("actual_yield","mean"),
        predicted=("predicted_yield","mean")
    )
    .reset_index()
)



In [36]:

fig_perf = px.line(
    region_perf,
    x="year",
    y=["actual","predicted"],
    markers=True,
    title=f"Actual vs Predicted Yield in {region_choice}"
)

fig_perf.show()